In [1]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 55 
num_classes = 3

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [2]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017-2018Rabi_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/2017-2018Rabi_VV1.csv', delimiter=',')
my_data0 = genfromtxt('./DataForBiplab/2017-2018Rabi_VH1.csv', delimiter=',')

In [4]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
data0 = my_data0[1:,2:]
print(data.shape)
print(data1.shape)
print(data0.shape)

(301, 27)
(301, 14)
(301, 14)


In [5]:
print(data.max())
print(data1.max())
print(data0.max())

0.935826175
0.280879855
0.076032609


In [6]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
data0 = np.array(data0)
print(data.shape)
print(data1.shape)
print(data0.shape)

(301, 27)
(301, 14)
(301, 14)


In [7]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataRabi2017-2018.csv', names=colnames)

In [8]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat

In [9]:
p = np.array(ctype)
print(p.shape)

(302,)


In [10]:
lbl1 = ctype
label = []
for i in range(1,302):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [11]:
label = np.array(label)
print(label.shape)

(301,)


In [12]:
print(data.shape)
print(data1.shape)
print(data0.shape)
data2 = np.concatenate((data,data1, data0), axis=1)
print(data2.shape)

(301, 27)
(301, 14)
(301, 14)
(301, 55)


In [13]:
x1=[]
x2=[]
x3=[]
l1=[]
l2=[]
l3=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)

print(l2.shape)

(142, 3)


In [14]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [15]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:240,:]
X_test = data2[241:,:]
y_train = label[0:240]
y_test = label[241:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(240, 55)
(60, 55)
(240,)
(60,)


In [16]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(240, 3)
(60, 3)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.83983636
Epoch: 2 cost = 0.75211747
Epoch: 3 cost = 0.67046285
Epoch: 4 cost = 0.59728879
Epoch: 5 cost = 0.53501576
Epoch: 6 cost = 0.48515401
Epoch: 7 cost = 0.44765761
Epoch: 8 cost = 0.42095995
Epoch: 9 cost = 0.40261551
Epoch: 10 cost = 0.39007113
Epoch: 11 cost = 0.38117786
Epoch: 12 cost = 0.37437024
Epoch: 13 cost = 0.36862779
Epoch: 14 cost = 0.36334848
Epoch: 15 cost = 0.35821267
Epoch: 16 cost = 0.35307209
Epoch: 17 cost = 0.34787244
Epoch: 18 cost = 0.34260670
Epoch: 19 cost = 0.33728986
Epoch: 20 cost = 0.33194694
Epoch: 21 cost = 0.32660639
Epoch: 22 cost = 0.32129483
Epoch: 23 cost = 0.31603096
Epoch: 24 cost = 0.31081996
Epoch: 25 cost = 0.30565036
Epoch: 26 cost = 0.30049565
Epoch: 27 cost = 0.29532062
Epoch: 28 cost = 0.29008918
Epoch: 29 cost = 0.28476842
Epoch: 30 cost = 0.27932847
Epoch: 31 cost = 0.27374017
Epoch: 32 cost = 0.26797267
Epoch: 33 cost = 0.26199101
Epoch: 34 cost = 0.25575536
Epoch: 35 cost = 0.24923147
Epoch: 36 cost = 0.24243299
E

Epoch: 319 cost = 0.00064146
Epoch: 320 cost = 0.00056813
Epoch: 321 cost = 0.00059940
Epoch: 322 cost = 0.00072668
Epoch: 323 cost = 0.00076684
Epoch: 324 cost = 0.00059565
Epoch: 325 cost = 0.00063441
Epoch: 326 cost = 0.00064984
Epoch: 327 cost = 0.00067647
Epoch: 328 cost = 0.00061813
Epoch: 329 cost = 0.00061602
Epoch: 330 cost = 0.00070954
Epoch: 331 cost = 0.00063384
Epoch: 332 cost = 0.00057749
Epoch: 333 cost = 0.00058126
Epoch: 334 cost = 0.00067393
Epoch: 335 cost = 0.00076863
Epoch: 336 cost = 0.00071358
Epoch: 337 cost = 0.00061634
Epoch: 338 cost = 0.00059395
Epoch: 339 cost = 0.00058068
Epoch: 340 cost = 0.00062406
Epoch: 341 cost = 0.00056913
Epoch: 342 cost = 0.00057952
Epoch: 343 cost = 0.00066421
Epoch: 344 cost = 0.00057173
Epoch: 345 cost = 0.00058138
Epoch: 346 cost = 0.00075984
Epoch: 347 cost = 0.00059786
Epoch: 348 cost = 0.00051084
Epoch: 349 cost = 0.00062371
Epoch: 350 cost = 0.00062041
Epoch: 351 cost = 0.00063080
Epoch: 352 cost = 0.00067603
Epoch: 353 cos

Epoch: 625 cost = 0.00033581
Epoch: 626 cost = 0.00030630
Epoch: 627 cost = 0.00033849
Epoch: 628 cost = 0.00026653
Epoch: 629 cost = 0.00026040
Epoch: 630 cost = 0.00031764
Epoch: 631 cost = 0.00032373
Epoch: 632 cost = 0.00032722
Epoch: 633 cost = 0.00033733
Epoch: 634 cost = 0.00035741
Epoch: 635 cost = 0.00040788
Epoch: 636 cost = 0.00034673
Epoch: 637 cost = 0.00026974
Epoch: 638 cost = 0.00027659
Epoch: 639 cost = 0.00032913
Epoch: 640 cost = 0.00030651
Epoch: 641 cost = 0.00026273
Epoch: 642 cost = 0.00024657
Epoch: 643 cost = 0.00028390
Epoch: 644 cost = 0.00027551
Epoch: 645 cost = 0.00029171
Epoch: 646 cost = 0.00024469
Epoch: 647 cost = 0.00037608
Epoch: 648 cost = 0.00047766
Epoch: 649 cost = 0.00028438
Epoch: 650 cost = 0.00026838
Epoch: 651 cost = 0.00028547
Epoch: 652 cost = 0.00032917
Epoch: 653 cost = 0.00034222
Epoch: 654 cost = 0.00025641
Epoch: 655 cost = 0.00025396
Epoch: 656 cost = 0.00037528
Epoch: 657 cost = 0.00033639
Epoch: 658 cost = 0.00027987
Epoch: 659 cos

Epoch: 937 cost = 0.00034822
Epoch: 938 cost = 0.00026136
Epoch: 939 cost = 0.00021460
Epoch: 940 cost = 0.00018461
Epoch: 941 cost = 0.00016465
Epoch: 942 cost = 0.00015205
Epoch: 943 cost = 0.00016335
Epoch: 944 cost = 0.00019117
Epoch: 945 cost = 0.00019639
Epoch: 946 cost = 0.00026483
Epoch: 947 cost = 0.00029674
Epoch: 948 cost = 0.00025370
Epoch: 949 cost = 0.00016205
Epoch: 950 cost = 0.00014496
Epoch: 951 cost = 0.00014944
Epoch: 952 cost = 0.00016963
Epoch: 953 cost = 0.00019030
Epoch: 954 cost = 0.00023667
Epoch: 955 cost = 0.00017769
Epoch: 956 cost = 0.00024101
Epoch: 957 cost = 0.00028459
Epoch: 958 cost = 0.00022430
Epoch: 959 cost = 0.00019209
Epoch: 960 cost = 0.00018947
Epoch: 961 cost = 0.00022646
Epoch: 962 cost = 0.00016514
Epoch: 963 cost = 0.00017826
Epoch: 964 cost = 0.00022540
Epoch: 965 cost = 0.00022832
Epoch: 966 cost = 0.00018933
Epoch: 967 cost = 0.00022094
Epoch: 968 cost = 0.00026438
Epoch: 969 cost = 0.00021159
Epoch: 970 cost = 0.00023357
Epoch: 971 cos

In [18]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
print(y_test.shape)

(60, 3)


In [20]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8333334


In [21]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
print(res1)
print(res2)
print(res3)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 1 2 1 1 2 2 2 2 2 1 1 1 1 2 1 1 2 1 2 2 2 2 1 1 1 2 2 2 1 1 1 1 2 2 1
 2 2]


In [22]:
# confusion matrix
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
confusion.append([x1,x2,x3])

confusion=np.array(confusion)
print(confusion)

[[110   9   1]
 [  8 129   5]
 [  0  19  20]]


In [39]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:210,:]
X_test = data2[210:,:]
y_train = label[0:210]
y_test = label[210:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(210, 55)
(91, 55)
(210,)
(91,)


In [40]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(210, 3)
(91, 3)


In [41]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.84176321
Epoch: 2 cost = 0.76664615
Epoch: 3 cost = 0.70025034
Epoch: 4 cost = 0.64463480
Epoch: 5 cost = 0.60050921
Epoch: 6 cost = 0.56683147
Epoch: 7 cost = 0.54131794
Epoch: 8 cost = 0.52139373
Epoch: 9 cost = 0.50491560
Epoch: 10 cost = 0.49043379
Epoch: 11 cost = 0.47712541
Epoch: 12 cost = 0.46460023
Epoch: 13 cost = 0.45270991
Epoch: 14 cost = 0.44141042
Epoch: 15 cost = 0.43068373
Epoch: 16 cost = 0.42050499
Epoch: 17 cost = 0.41083558
Epoch: 18 cost = 0.40162916
Epoch: 19 cost = 0.39283972
Epoch: 20 cost = 0.38442786
Epoch: 21 cost = 0.37636237
Epoch: 22 cost = 0.36861752
Epoch: 23 cost = 0.36116647
Epoch: 24 cost = 0.35397325
Epoch: 25 cost = 0.34698912
Epoch: 26 cost = 0.34015744
Epoch: 27 cost = 0.33342824
Epoch: 28 cost = 0.32677299
Epoch: 29 cost = 0.32018601
Epoch: 30 cost = 0.31367009
Epoch: 31 cost = 0.30721837
Epoch: 32 cost = 0.30081099
Epoch: 33 cost = 0.29443059
Epoch: 34 cost = 0.28808793
Epoch: 35 cost = 0.28184425
Epoch: 36 cost = 0.27575978
E

Epoch: 310 cost = 0.00170540
Epoch: 311 cost = 0.00181257
Epoch: 312 cost = 0.00180365
Epoch: 313 cost = 0.00179652
Epoch: 314 cost = 0.01465123
Epoch: 315 cost = 0.00496080
Epoch: 316 cost = 0.00165074
Epoch: 317 cost = 0.00163524
Epoch: 318 cost = 0.00163001
Epoch: 319 cost = 0.00161138
Epoch: 320 cost = 0.00156636
Epoch: 321 cost = 0.00153410
Epoch: 322 cost = 0.00151532
Epoch: 323 cost = 0.00152541
Epoch: 324 cost = 0.00157149
Epoch: 325 cost = 0.00161445
Epoch: 326 cost = 0.00153640
Epoch: 327 cost = 0.00147280
Epoch: 328 cost = 0.00152022
Epoch: 329 cost = 0.00158783
Epoch: 330 cost = 0.00150478
Epoch: 331 cost = 0.00147207
Epoch: 332 cost = 0.00154825
Epoch: 333 cost = 0.00154659
Epoch: 334 cost = 0.00145383
Epoch: 335 cost = 0.00151028
Epoch: 336 cost = 0.00161802
Epoch: 337 cost = 0.00152340
Epoch: 338 cost = 0.00144738
Epoch: 339 cost = 0.00147528
Epoch: 340 cost = 0.00153596
Epoch: 341 cost = 0.00151906
Epoch: 342 cost = 0.00159133
Epoch: 343 cost = 0.00188280
Epoch: 344 cos

Epoch: 620 cost = 0.00042862
Epoch: 621 cost = 0.00041762
Epoch: 622 cost = 0.00041910
Epoch: 623 cost = 0.00049118
Epoch: 624 cost = 0.00050822
Epoch: 625 cost = 0.00046751
Epoch: 626 cost = 0.00046630
Epoch: 627 cost = 0.00037439
Epoch: 628 cost = 0.00044937
Epoch: 629 cost = 0.00044931
Epoch: 630 cost = 0.00039258
Epoch: 631 cost = 0.00039011
Epoch: 632 cost = 0.00042773
Epoch: 633 cost = 0.00050773
Epoch: 634 cost = 0.00040344
Epoch: 635 cost = 0.00033786
Epoch: 636 cost = 0.00041497
Epoch: 637 cost = 0.00041900
Epoch: 638 cost = 0.00041586
Epoch: 639 cost = 0.00048057
Epoch: 640 cost = 0.00044302
Epoch: 641 cost = 0.00038841
Epoch: 642 cost = 0.00040093
Epoch: 643 cost = 0.00050885
Epoch: 644 cost = 0.00048135
Epoch: 645 cost = 0.00035837
Epoch: 646 cost = 0.00041262
Epoch: 647 cost = 0.00040546
Epoch: 648 cost = 0.00036772
Epoch: 649 cost = 0.00045806
Epoch: 650 cost = 0.00035982
Epoch: 651 cost = 0.00042390
Epoch: 652 cost = 0.00032704
Epoch: 653 cost = 0.00033913
Epoch: 654 cos

Epoch: 935 cost = 0.00024229
Epoch: 936 cost = 0.00027137
Epoch: 937 cost = 0.00032676
Epoch: 938 cost = 0.00031811
Epoch: 939 cost = 0.00034226
Epoch: 940 cost = 0.00022803
Epoch: 941 cost = 0.00020967
Epoch: 942 cost = 0.00023199
Epoch: 943 cost = 0.00021779
Epoch: 944 cost = 0.00020422
Epoch: 945 cost = 0.00023350
Epoch: 946 cost = 0.00026056
Epoch: 947 cost = 0.00032346
Epoch: 948 cost = 0.00031617
Epoch: 949 cost = 0.00021180
Epoch: 950 cost = 0.00028814
Epoch: 951 cost = 0.00032457
Epoch: 952 cost = 0.00021100
Epoch: 953 cost = 0.00027048
Epoch: 954 cost = 0.00022720
Epoch: 955 cost = 0.00021166
Epoch: 956 cost = 0.00025317
Epoch: 957 cost = 0.00024675
Epoch: 958 cost = 0.00029597
Epoch: 959 cost = 0.00028321
Epoch: 960 cost = 0.00023706
Epoch: 961 cost = 0.00027311
Epoch: 962 cost = 0.00025856
Epoch: 963 cost = 0.00020066
Epoch: 964 cost = 0.00021026
Epoch: 965 cost = 0.00025015
Epoch: 966 cost = 0.00030277
Epoch: 967 cost = 0.00025568
Epoch: 968 cost = 0.00024462
Epoch: 969 cos

In [42]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8351649
